# Web Reader Test

In [ ]:
from web_reader import web_reader
account_setting = {"user-data-dir":"path1",
"--profile-directory":"path2"}
web_reader = web_reader("https://google.com",account_setting= account_setting)
web_reader.read()[0]

# Markdown: generate functions description test

In [ ]:
from prompting import generate_functions_descriptions,function_data
print(generate_functions_descriptions(function_data))

# Plan generating Test

In [ ]:
from prompting import plan_prompt
import requests
import json
url = "http://localhost:11434/api/generate"
model = 'qwen2.5:3b'
final_prompt = plan_prompt + "you start from current_url:https://google.com \n Your task:Open ucsd.edu"
response = requests.post(url, json={
                "model": model,
                "prompt": final_prompt,
                "streaming": False,
                "options": {
                    "temperature": 0.5
                },
            }, stream=False)
for i in response.iter_lines():
    print(json.loads(i.decode('utf-8'))["response"]
        ,end = "")


In [ ]:
from prompting import plan_prompt,generate_elements_prompt
import requests
import json
from web_reader import web_reader
account_setting = {"user-data-dir":"/Users/gongzhenghao/Desktop/Google_copy",
"--profile-directory":"Profile 3"}
web_reader = web_reader("https://google.com",account_setting= account_setting)
str1,record = generate_elements_prompt(web_reader.read()[0])
str2= web_reader.read()[2]

In [ ]:
from prompting import plan_prompt,summary_prompt,summerization
import requests
import json
from web_reader import web_reader
account_setting = {"user-data-dir":"/Users/gongzhenghao/Desktop/Google_copy",
"--profile-directory":"Profile 3"}

url = "http://localhost:11434/api/generate"
model = 'qwen2.5:3b'

# final_prompt = summary_prompt.replace("{web_content}",str1).replace("{text}",str2) + ""
# response = requests.post(url, json={
#                 "model": model,
#                 "prompt": final_prompt,
#                 "streaming": False,
#                 "options": {
#                     "temperature": 0.5
#                 },
#             }, stream=False)
# result = ""
# for i in response.iter_lines():
#     result += json.loads(i.decode('utf-8'))["response"]
summerization(str1,str2,url,model)

In [ ]:
llm_url = "http://localhost:11434/api/generate"
model = 'qwen2.5:3b'
url = "https://www.google.com/search?q=ucsd&oq=ucsd&gs_lcrp=EgZjaHJvbWUqDQgAEAAY4wIYsQMYgAQyDQgAEAAY4wIYsQMYgAQyBggBEEUYPDIKCAIQLhixAxiABDIGCAMQRRg8MgYIBBBFGEEyBggFEEUYPDIGCAYQRRhBMgYIBxBFGEHSAQgzNTUwajBqN6gCALACAA&sourceid=chrome&ie=UTF-8"
from web_reader import web_reader
account_setting = {"user-data-dir":"path1",
"--profile-directory":"path2"}
web_reader = web_reader(url,account_setting= account_setting)
str1,record = generate_elements_prompt(web_reader.read()[0])
str2= web_reader.read()[2]
print(len(record))
for r in record:
    final_prompt = "Based on the task, determine if this elements is useful for or related to this task, task:search for ucsd,\n web_elements:" + r + "you should only response yes or no"
    response = requests.post(llm_url, json={
                    "model": model,
                    "prompt": final_prompt,
                    "streaming": False,
                    "options": {
                        "temperature": 0.5
                    },
                }, stream=False)
    result = ""
    for i in response.iter_lines():
        result += json.loads(i.decode('utf-8'))["response"]
    if result == "yes":
        print(r,result)


In [ ]:
import os

# 定义代码所在的目录
base_path = '/Users/gongzhenghao/Desktop/Web_Agent'

# 在指定目录下递归统计所有的 .py 文件，以列表形式返回
def collect_files(dir_path):
    return [
        os.path.join(parent, filename)
        for parent, _, filenames in os.walk(dir_path)
        for filename in filenames if filename.endswith('.py')
    ]

# 计算单个文件内的代码行数
def calc_linenum(file_path):
    code_num = 0      # 当前文件代码行数计数变量
    blank_num = 0     # 当前文件空行数计数变量
    annotate_num = 0  # 当前文件注释行数计数变量

    with open(file_path, 'r', encoding='utf-8') as fp:
        for line in fp:
            line = line.strip()
            if not line:  # 空行
                blank_num += 1
            elif line.startswith('#'):  # 注释行
                annotate_num += 1
            else:  # 代码行
                code_num += 1

    # 返回代码行数，空行数，注释行数
    return code_num, blank_num, annotate_num

# 统计目录中所有 .py 文件的代码行数
def count_code_lines(base_dir):
    files = collect_files(base_dir)
    total_code_num = 0    # 总代码行数
    total_blank_num = 0   # 总空行数
    total_annotate_num = 0  # 总注释行数

    print(f"{'文件路径':<60} {'代码行数':<10} {'空行数':<10} {'注释行数':<10}")
    print("=" * 90)

    for file_path in files:
        code_num, blank_num, annotate_num = calc_linenum(file_path)
        total_code_num += code_num
        total_blank_num += blank_num
        total_annotate_num += annotate_num
        print(f"{file_path:<60} {code_num:<10} {blank_num:<10} {annotate_num:<10}")

    print("=" * 90)
    print(f"代码总行数： {total_code_num}")
    print(f"空行总行数： {total_blank_num}")
    print(f"注释行总行数： {total_annotate_num}")

# 执行统计
count_code_lines(base_path)


In [ ]:
from prompting import reflection_prompt,generate_reflection,generate_elements_prompt,generate_functions_descriptions,function_data
import requests
import json
from web_reader import web_reader
model = "qwen2.5:3b"
llm_url = "http://localhost:11434/api/generate"
url = "https://www.google.com"
account_setting = {"user-data-dir":"path1",
"--profile-directory":"path2"}
web_reader = web_reader("https://google.com",account_setting= account_setting)
total = web_reader.read()
web_content,record = generate_elements_prompt(total[0])
text = total[2]
subtask = "search for ucsd"
action = "typing(7,'ucsd')"
function_str = generate_functions_descriptions(function_data)
insert_place0 = ""
insert_place1 = ""
insert_place2 = ""
print("start_calling")
generate_reflection(subtask,function_str,action,web_content,text,llm_url,model,"{id=7, type = textarea, innertext = 搜索}",insert_place0,insert_place1,insert_place2)